In [38]:
#Importing packages we want
import pandas as pd
import numpy as np
import collections
import re

In [2]:
#Reading data
df = pd.read_csv('twitter.csv')

/Users/air/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
1,1,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
2,2,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
3,3,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
4,4,906000000000000000,10_GOP,"Dan Bongino: ""Nobody trolls liberals better th...",Unknown,English,10/1/2017 2:47,10/1/2017 2:47,1050,9644,...,Right,0,RightTroll,0,905874659358453760,914320835325853696,http://twitter.com/905874659358453760/statuses...,https://twitter.com/FoxNews/status/91423949678...,NaN,NaN


In [4]:
# Dropping some columns which may not be significant for our further analysis
df.drop(columns = ['Unnamed: 0', 'external_author_id', 'article_url', 'harvested_date', 'tweet_id'], axis = 1, inplace = True)

In [5]:
df.isnull().sum()

author                    0
content                   1
region                 8818
language                  0
publish_date              0
following                 0
followers                 0
updates                   0
post_type           1648624
account_type              0
retweet                   0
account_category          0
new_june_2018             0
alt_external_id           0
tco1_step1           845971
tco2_step1          2235089
tco3_step1          2931206
dtype: int64

In [6]:
# Since there are too many null values in ['tco2_step1', 'tco3_step1'] columns, we can drop them 
df.drop(columns = ['tco2_step1', 'tco3_step1'], axis = 1, inplace = True)

In [7]:
# Getting the percentage of how many tweets come from the United States
print(df.region.unique())
print(len(df[df['region'] == 'United States']) / len(df))

['Unknown' nan 'United States' 'Italy' 'United Arab Emirates' 'Japan'
 'Israel' 'Azerbaijan' 'Egypt' 'United Kingdom' 'Russian Federation'
 'Turkey' 'Iraq' 'Germany' 'France' 'Ukraine' 'Serbia' 'Hong Kong'
 'Austria' 'Belarus' 'Malaysia' 'Spain' 'Samoa' 'India' 'Afghanistan'
 'Saudi Arabia' 'Iran, Islamic Republic of' 'Mexico' 'Canada' 'Greece'
 'Czech Republic' 'Finland' 'Latvia' 'Estonia' 'Sweden' 'Denmark'
 'Switzerland']
0.6946931069993069


In [8]:
# Getting the percentage of how many tweets are English
print(df.language.unique())
print(len(df[df['language'] == 'English']) / len(df))

['English' 'Russian' 'Serbian' 'Ukrainian' 'Tagalog (Filipino)' 'Albanian'
 'Italian' 'Romanian' 'Spanish' 'Catalan' 'German' 'Estonian' 'French'
 'Norwegian' 'Vietnamese' 'Dutch' 'Arabic' 'Uzbek' 'Bulgarian'
 'Macedonian' 'Farsi (Persian)' 'Turkish' 'LANGUAGE UNDEFINED' 'Czech'
 'Somali' 'Lithuanian' 'Croatian' 'Slovak' 'Icelandic' 'Slovenian'
 'Japanese' 'Indonesian' 'Pushto' 'Hungarian' 'Finnish' 'Latvian'
 'Portuguese' 'Danish' 'Swedish' 'Malay' 'Polish' 'Korean' 'Hebrew' 'Urdu'
 'Kurdish' 'Hindi' 'Greek' 'Simplified Chinese' 'Thai' 'Bengali'
 'Traditional Chinese' 'Gujarati' 'Kannada' 'Tamil' 'Telugu' 'Malayalam']
0.7185057663992267


In [9]:
df = df[df['language'] == 'English']
df = df[df['region'] == 'United States']

In [10]:
def get_mdy(string):
    return string.split()[0]
def get_hm(string):
    return string.split()[1]

In [11]:
df['pb_mdy_info'] = df.publish_date.apply(get_mdy)
df['pb_hm_info'] = df.publish_date.apply(get_hm)

In [12]:
def get_month(string):
    return string.split('/')[0]
def get_date(string):
    return string.split('/')[1]
def get_hour(string):
    return string.split(':')[0]

In [13]:
df['pb_month'] = df.pb_mdy_info.apply(get_month)
df['pb_date'] = df.pb_mdy_info.apply(get_date)
df['pb_hour'] = df.pb_hm_info.apply(get_hour)

In [14]:
df.head()

,author,content,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tco1_step1,pb_mdy_info,pb_hm_info,pb_month,pb_date,pb_hour
423,1D_NICOLE_,#FoodPoisoning is not a joke! #Walmart #KochFa...,United States,English,11/26/2015 22:20,48,40,394,RETWEET,Koch,1,Fearmonger,0,2535564756,https://twitter.com/Ina_Malone/status/67000248...,11/26/2015,22:20,11,26,22
424,1D_NICOLE_,Thanks God that #turkey i ate few days ago was...,United States,English,11/26/2015 22:35,48,40,395,RETWEET,Koch,1,Fearmonger,0,2535564756,https://twitter.com/Willie_BigN/status/6700043...,11/26/2015,22:35,11,26,22
425,1D_NICOLE_,"I hurt easy, I just don't show it. You can hur...",United States,English,5/25/2015 12:38,59,51,354,NaN,Koch,0,Fearmonger,0,2535564756,NaN,5/25/2015,12:38,5,25,12
427,1D_NICOLE_,"who's got the worst make up ever, don't tell m...",United States,English,5/25/2015 12:38,59,51,352,NaN,Koch,0,Fearmonger,0,2535564756,NaN,5/25/2015,12:38,5,25,12
428,1D_NICOLE_,jade doesn`t love her! https://t.co/fEOsiKho7C,United States,English,5/25/2015 12:39,59,51,357,NaN,Koch,0,Fearmonger,0,2535564756,https://vine.co/v/OIPJQ21bwg9,5/25/2015,12:39,5,25,12


In [15]:
# Getting hash tags(may have better way)

In [16]:
contents = df['content'].tolist()

In [17]:
pattern = '#\w+' 

In [31]:
def get_hash_tags(pattern, contents):
    word_list = []
    for sentence in contents:
        if isinstance(sentence, str):
            word_list += re.findall(pattern, sentence)
    return word_list

In [32]:
#Test 
test = df['content'].head()
get_hash_tags(pattern, test)

['#FoodPoisoning', '#Walmart', '#KochFarms', '#turkey', '#KochFarms']

In [33]:
hash_tags = get_hash_tags(pattern, contents)

In [34]:
hash_tags

['#FoodPoisoning',
 '#Walmart',
 '#KochFarms',
 '#turkey',
 '#KochFarms',
 '#JasonAldean',
 '#hungry',
 '#DumbGenieWishes',
 '#DumbGenieWishes',
 '#YoLandi',
 '#DumbGenieWishes',
 '#petition',
 '#NoGunsForCriminals',
 '#black',
 '#NoGunsForCriminals',
 '#Raiders',
 '#Carr',
 '#RaiderNation',
 '#Raiders',
 '#MikeAndMolly',
 '#CaitlynJenner',
 '#ConversationWithAlanis',
 '#HOLYPOOPICANPUTT',
 '#WillSmith',
 '#DearSugarRadio',
 '#ConversationWithAlanis',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#VegasGOPDebate',
 '#Cruz',
 '#VegasGOPDebate',
 '#Bush',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#CNNDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#CNNDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#VegasGOPDebate',
 '#CNNDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#CNNDebate',
 '#GOPDebate',
 '#VegasGOPDebate',
 '#GOPDebate',
 '#VegasGOPDebate',

In [35]:
def count_hashtags(hash_tags):
    tag_to_num = collections.defaultdict(int)
    for hashtag in hash_tags:
        tag_to_num[hashtag] += 1 
    return tag_to_num

In [39]:
tag_to_num = count_hashtags(hash_tags)

In [40]:
tag_to_num

defaultdict(int,
            {'#FoodPoisoning': 772,
             '#Walmart': 960,
             '#KochFarms': 3748,
             '#turkey': 885,
             '#JasonAldean': 40,
             '#hungry': 10,
             '#DumbGenieWishes': 280,
             '#YoLandi': 1,
             '#petition': 93,
             '#NoGunsForCriminals': 432,
             '#black': 237,
             '#Raiders': 21,
             '#Carr': 2,
             '#RaiderNation': 32,
             '#MikeAndMolly': 3,
             '#CaitlynJenner': 75,
             '#ConversationWithAlanis': 13,
             '#HOLYPOOPICANPUTT': 2,
             '#WillSmith': 11,
             '#DearSugarRadio': 2,
             '#GOPDebate': 3864,
             '#VegasGOPDebate': 2320,
             '#Cruz': 165,
             '#Bush': 66,
             '#CNNDebate': 220,
             '#Power100': 2,
             '#WCW': 334,
             '#wordsofwisdom': 5,
             '#quoteoftheday': 37,
             '#IslamKills': 3955,
            

In [41]:
len(tag_to_num)

85146

In [42]:
df.isnull().sum()

author                    0
content                   1
region                    0
language                  0
publish_date              0
following                 0
followers                 0
updates                   0
post_type           1023675
account_type              0
retweet                   0
account_category          0
new_june_2018             0
alt_external_id           0
tco1_step1           722589
pb_mdy_info               0
pb_hm_info                0
pb_month                  0
pb_date                   0
pb_hour                   0
dtype: int64

In [43]:
df.post_type.unique()

array(['RETWEET', nan, 'QUOTE_TWEET'], dtype=object)

In [44]:
# Maybe we don't have to deal with this column yet.